In [1]:
%matplotlib inline
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#reading the image
img = cv.imread('Fig_1_a.png')
#converting the image to grayscale
img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
#reshaping image
img_r = img.reshape((-1,1))

#printing dimensions of the image
r,c = img.shape
print(f"Rows: {r} \nColumns: {c}")


Rows: 202 
Columns: 217


In [3]:
#converting the type of the reshaped image to float32
img_r = np.float32(img_r)

In [4]:
#Applying image segmentation using k-means clustering
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
#k is the number of segements or clusters
k = 3 
ret,label,center=cv.kmeans(img_r,k,None,criteria,10,cv.KMEANS_PP_CENTERS)

In [5]:
#converting the type of center values to uint8
center = np.uint8(center)
res = center[label.flatten()]
#reshaping it back to the original size
res2 = res.reshape((img.shape))

In [6]:
cv.imshow('segemented',res2)
cv.imwrite('segmented8.jpg',res2)
cv.waitKey(0)
cv.destroyAllWindows()

In [7]:
cluster_counts = np.bincount(label.flatten())

print("Number of pixels in each segmentation (cluster):")
for i, count in enumerate(cluster_counts):
  print(f"Cluster {i+1}: {count}")

Number of pixels in each segmentation (cluster):
Cluster 1: 17026
Cluster 2: 5661
Cluster 3: 21147


In [8]:
values = center.flatten()
print(values)

#retrieving the least value for umbra and second least for penumbra and storing them
umb = np.min(values)
values = [val for val in values if val > umb]
penu = np.min(values)
print("maximum average value for umbra: ",umb)
print("maximum average value for penumbra: ",penu) 

[172  10 110]
maximum average value for umbra:  10
maximum average value for penumbra:  110


In [10]:
seg = cv.imread('segmented8.jpg')
seg_g = cv.cvtColor(seg,cv.COLOR_BGR2GRAY)

cv.imshow('seg',seg_g)

In [11]:
#creating a image of white
plain= np.full((r, c, 3), 255, dtype='uint8')

In [12]:
#calculating the correction factor for penumbra
maxf = np.max(values)
correction_factor_p = int((maxf-penu) / 2)
print("Correction factor for penumbra: ",correction_factor_p)

Correction factor for penumbra:  31


In [13]:
#calculating the correction factor for umbra
correction_factor_u = int((penu-umb)/2)
print("Correction factor for umbra: ",correction_factor_u)

Correction factor for umbra:  50


In [14]:
#retrieving values of penumbra
penumbra=cv.cvtColor(plain, cv.COLOR_BGR2GRAY)
count_p =0  

for i in range(r):
    for j in range(c):
        if seg_g[i,j] >umb+correction_factor_u:
            if seg_g[i,j] <=penu+correction_factor_p:
                penumbra[i,j] = seg_g[i,j]
                count_p += 1  

                
print('Total number of pixels in penumbra: ',count_p)               
cv.imshow('penum',penumbra)
cv.waitKey(0)         

Total number of pixels in penumbra:  21147


48

In [15]:
#retrieving values of penumbra
count_u =0
umbra=cv.cvtColor(plain, cv.COLOR_BGR2GRAY)

for i in range(r):
    for j in range(c):
        if seg_g[i,j]<=umb+correction_factor_u:
            umbra[i,j] = seg_g[i,j]
            count_u += 1

cv.imshow('umbra',umbra)
cv.waitKey(0) 
print('Total number of pixels in umbra: ',count_u)
print('Ratio of penumbra over umbra: ',count_p/count_u)
print('Ratio of umbra over penumbra: ',count_u/count_p)

Total number of pixels in umbra:  5661
Ratio of penumbra over umbra:  3.735559088500265
Ratio of umbra over penumbra:  0.2676975457511704
